# Examining VIC-WRF data and exporting

In [1]:
%matplotlib widget

import __init__
import scripts.config as config
import numpy as np
import pandas as pd
import tempfile
import datetime
from sklearn.svm import SVR
from natsort import natsorted
import geopandas as gpd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from matplotlib.font_manager import FontProperties
import seaborn as sns
# import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib
import HydroErr as he
import os

In [2]:
# Plotting parameters

XSMALL_SIZE = 6
SMALL_SIZE = 7
MEDIUM_SIZE = 9
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the figure title
plt.rcParams['figure.dpi'] = 140

In [6]:
# Import and format observed data (2003-2007 runoff)

input_dir = config.velma_data
results_dir = config.data_path.parents[0] / 'results' / 'ellsworth_baseline_03_07_21'

runoff_start = pd.to_datetime('01-01-2003')
runoff_end = pd.to_datetime('12-31-2007')
nse_start = pd.to_datetime('01-01-2004')
nse_end = pd.to_datetime('12-31-2007')

vicwrf_dir = config.data_path / 'precip' / 'VIC_WRF_EllsworthCr'
vicwrf_file = 'flux_46.40625_-123.90625'
vicwrf_cols = ["YEAR","MONTH","DAY","HOUR","OUT_PREC","OUT_PET_SHORT",
            "OUT_SWE","OUT_EVAP","OUT_RUNOFF","OUT_BASEFLOW",
            "OUT_SOIL_MOIST0", "OUT_SOIL_MOIST1","OUT_SOIL_MOIST2"]
pnnl_hist = pd.read_csv(vicwrf_dir / 'pnnl_historical' / 'sim0' / vicwrf_file, 
                        names=vicwrf_cols, delimiter='\t', parse_dates={'DATE': ['YEAR', 'MONTH', 'DAY']},
                        index_col=0)
pnnl_hist = pnnl_hist[(pnnl_hist.index >= nse_start) & (pnnl_hist.index <= nse_end)]

# Runoff observations
runoff_path = input_dir / 'runoff' / 'ellsworth_Q_2003_2007_dummy.csv'
runoff_obs = pd.read_csv(runoff_path, names=['runoff_obs'])
runoff_obs.index = pd.date_range(runoff_start, runoff_end)
runoff_obs['doy'], runoff_obs['year'] = runoff_obs.index.dayofyear, runoff_obs.index.year
runoff_obs = runoff_obs[(runoff_obs.index >= nse_start) & (runoff_obs.index <= nse_end)]

# Runoff quality codes
flow_path = config.streamflow
quality = pd.read_csv(flow_path, usecols=['Date', 'Quality'], parse_dates=True, index_col=0)
quality = quality[(quality.index >= nse_start) & (quality.index <= nse_end)]

# Precipitation
precip_path = input_dir / 'precip' / 'PRISM_gauge_avg_ppt_2003_2019.csv'
forcing_start = pd.to_datetime('01-01-2003')
forcing_end = pd.to_datetime('12-31-2019')                     
precip = pd.read_csv(precip_path, names=['precip'])
precip.index = pd.date_range(forcing_start, forcing_end)
precip['doy'], precip['year'] = precip.index.dayofyear, precip.index.year
precip = precip[(precip.index >= nse_start) & (precip.index <= nse_end)]

# Air temperature
temp_path = input_dir / 'temp' / 'ellsworth_temp_2003_2019.csv'
temp = pd.read_csv(temp_path, names=['temp'])
temp.index = pd.date_range(forcing_start, forcing_end)
temp['doy'], temp['year'] = temp.index.dayofyear, temp.index.year
temp = temp[(temp.index >= nse_start) & (temp.index <= nse_end)]

# Import VELMA outputs
velma_results = pd.read_csv(results_dir / 'DailyResults.csv')

# Format datetime of results
jday_pad = velma_results['Day'].apply(lambda x: str(x).zfill(3))
str_year = velma_results['Year'].apply(lambda x: str(x))
velma_results['year_jday'] = str_year + jday_pad
velma_results.index = pd.to_datetime(velma_results['year_jday'], format='%Y%j')
velma_results = velma_results[(velma_results.index >= nse_start) & (velma_results.index <= nse_end)]

In [ ]:
# Make pivot tables

## Plotting

## Exporting

In [8]:
driver_start = pd.to_datetime('01-01-2003')
driver_end = pd.to_datetime('12-31-2019')

pnnl_hist_export = pd.read_csv(vicwrf_dir / 'pnnl_historical' / 'sim0' / vicwrf_file, 
                        names=vicwrf_cols, delimiter='\t', parse_dates={'DATE': ['YEAR', 'MONTH', 'DAY']},
                        index_col=0)
pnnl_hist_export = pnnl_hist[(pnnl_hist.index >= driver_start) & (pnnl_hist.index <= driver_end)]

# Precipitation
outfile = input_dir / 'precip' / 'vic_wrf_ppt_03_19.csv'
ppt_out = pnnl_hist_export.loc[:, 'OUT_PREC']
ppt_out.to_csv(outfile, header=False, index=False)

# Air temperature
outfile = input_dir / 'precip' / 'vic_wrf_temp_03_19.csv'
temp_out = xxxx
temp_out.to_csv(outfile, header=False, index=False)
# ppt_out = pnnl_hist_export.loc[:, 'OUT_PREC']

NameError: name 'xxxx' is not defined